In [1]:
from pathlib import Path
import random
import shutil

# Reproducibility
random.seed(42)

# Paths
SRC_DIR = Path("../../../data/combined_annotated_2class")
OUT_DIR = Path("../../../data/rdd2class_yolo")

IMG_EXT = ".jpg"
LBL_EXT = ".txt"

In [2]:
images = sorted(SRC_DIR.glob(f"*{IMG_EXT}"))
labels = sorted(SRC_DIR.glob(f"*{LBL_EXT}"))

print(f"Images found: {len(images)}")
print(f"Labels found: {len(labels)}")

assert len(images) == len(labels), "Mismatch between images and labels!"

Images found: 25600
Labels found: 25600


In [3]:
split_ratio = 0.8

random.shuffle(images)
split_idx = int(len(images) * split_ratio)

train_images = images[:split_idx]
val_images = images[split_idx:]

print(f"Train images: {len(train_images)}")
print(f"Val images: {len(val_images)}")

Train images: 20480
Val images: 5120


In [4]:
for p in [
    "images/train",
    "images/val",
    "labels/train",
    "labels/val",
]:
    (OUT_DIR / p).mkdir(parents=True, exist_ok=True)

In [5]:
def copy_split(img_list, split):
    for img in img_list:
        lbl = img.with_suffix(LBL_EXT)

        # Copy image
        shutil.copy(img, OUT_DIR / f"images/{split}/{img.name}")

        dst_lbl = OUT_DIR / f"labels/{split}/{lbl.name}"

        # If label exists, copy it
        if lbl.exists():
            shutil.copy(lbl, dst_lbl)
        else:
            # Otherwise create empty label file
            dst_lbl.write_text("")

In [6]:
copy_split(train_images, "train")
copy_split(val_images, "val")

print("✅ Dataset split completed (missing labels handled)")

✅ Dataset split completed (missing labels handled)


In [7]:
train_imgs = list((OUT_DIR / "images/train").glob("*.jpg"))
train_lbls = list((OUT_DIR / "labels/train").glob("*.txt"))

val_imgs = list((OUT_DIR / "images/val").glob("*.jpg"))
val_lbls = list((OUT_DIR / "labels/val").glob("*.txt"))

print("Train:", len(train_imgs), "images |", len(train_lbls), "labels")
print("Val  :", len(val_imgs), "images |", len(val_lbls), "labels")

Train: 20480 images | 20480 labels
Val  : 5120 images | 5120 labels


In [8]:
DATA_YAML = OUT_DIR / "rdd2class.yaml"

DATA_YAML.write_text(
"""path: rdd2class_yolo

train: images/train
val: images/val

nc: 2
names:
  0: crack
  1: pothole
"""
)

print(f"📄 data.yaml created at {DATA_YAML}")

📄 data.yaml created at ../../../data/rdd2class_yolo/rdd2class.yaml


In [9]:
from ultralytics import YOLO

PRETRAINED_MODEL = "../../../runs/three_country_training/road_defect_binary/yolov8n8/weights/best.pt"

model = YOLO(PRETRAINED_MODEL)


save_dir = "../../../runs/yolov8n8_rdd2022_2class"

In [10]:
from pathlib import Path
print(Path("../../../data/rdd2class_yolo").resolve())

/home/saber/GitHub/road_anomaly_detection/data/rdd2class_yolo


In [15]:
model.train(
    data="/home/saber/GitHub/road_anomaly_detection/data/rdd2class_yolo/rdd2class.yaml",
    epochs=70,
    imgsz=640,
    batch=8,
    freeze=10,
    name="yolov8n8_rdd2022_2class"
)

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7799MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/saber/GitHub/road_anomaly_detection/data/rdd2class_yolo/rdd2class.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=70, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../../..

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f445c1d9880>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [16]:
from ultralytics import YOLO

PRETRAINED_MODEL = "../../../runs/three_country_training/road_defect_binary/yolov8n8/weights/best.pt"

model = YOLO(PRETRAINED_MODEL)


save_dir = "../../../runs/yolov8n8_rdd2022_2class_256"

In [17]:
model.train(
    data="/home/saber/GitHub/road_anomaly_detection/data/rdd2class_yolo/rdd2class.yaml",
    epochs=70,
    imgsz=256,
    batch=16,
    freeze=10,
    name="yolov8n8_rdd2022_2class_256"
)

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7799MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/saber/GitHub/road_anomaly_detection/data/rdd2class_yolo/rdd2class.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=70, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../../.

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f445c09ffe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804